In [ ]:
import os
import re
import folium
import pandas as pd
import geopandas as gpd
import osmnx as ox
from pathlib import Path
from urllib.request import urlretrieve
from zipfile import ZipFile

Download files from https://download.geofabrik.de/:

In [11]:
ab_url = "https://download.geofabrik.de/north-america/canada/alberta-latest-free.shp.zip"
urlretrieve(ab_url, "../data/ab-osm.zip")

('../data/ab-osm.zip', <http.client.HTTPMessage at 0x731289b3bed0>)

In [ ]:
with ZipFile("../data/ab-osm.zip", "r") as zObject:
    zObject.extractall(path="../data/raw/osm-data")

os.unlink("../data/ab-osm.zip")

In [18]:
osm = list()
p = Path("../data/raw/osm-data/")
for i in p.glob("*.shp"):
    df = gpd.read_file(i, engine="pyogrio")
    key_name = re.sub("gis_osm_|_a|_free_1|.shp", "", i.name)
    df["key"] = key_name
    osm.append(df)

osm = pd.concat(osm)
osm = gpd.GeoDataFrame(osm)
osm = osm.loc[:, ["key", "fclass", "name", "geometry"]]
osm = osm.rename({"fclass": "value"}, axis=1)
osm.head()

,key,value,name,geometry
0,landuse,park,Nose Hill Park,"POLYGON ((-114.13938 51.10938, -114.13937 51.1..."
1,landuse,park,West Confederation Park,"POLYGON ((-114.11515 51.08155, -114.11496 51.0..."
2,landuse,park,Cochrane Ranche,"POLYGON ((-114.48593 51.19858, -114.48541 51.1..."
3,landuse,park,Mitford Park,"POLYGON ((-114.49651 51.19028, -114.49649 51.1..."
4,landuse,park,W.D. Kerfoot Park,"POLYGON ((-114.47413 51.19595, -114.47371 51.1..."


In [27]:
try:
    os.makedirs("../data/processed")
except FileExistsError:
    pass

osm.to_file("../data/processed/osm.gpkg", driver="GPKG")

In [23]:
osm = gpd.read_file("../data/processed/osm.gpkg", engine="pyogrio")
osm = osm.loc[osm.geometry.geom_type == "Polygon"]
osm.head()

,key,value,name,geometry
0,transport,apron,None,"POLYGON ((-113.07905 53.72801, -113.07896 53.7..."
1,transport,apron,None,"POLYGON ((-113.59230 53.30792, -113.58939 53.3..."
2,transport,airport,Peace River Airport,"POLYGON ((-117.46573 56.21861, -117.46557 56.2..."
3,transport,airport,Edmonton International Airport,"POLYGON ((-113.61016 53.30148, -113.61005 53.3..."
4,transport,apron,None,"POLYGON ((-112.78989 49.63371, -112.78983 49.6..."


In [ ]:
schema = pd.read_csv("../data/tables/osm-lulc.csv")
schema = schema.dropna(subset="lulc")
schema.head()

In [27]:
osm_lulc = osm.merge(schema, on=["key", "value"], how="inner")
osm_lulc.head()

,key,value,name,geometry,lulc
0,landuse,retail,None,"POLYGON ((-114.15886 51.08555, -114.15862 51.0...",urban_high
1,landuse,retail,Chinook Centre,"POLYGON ((-114.07618 50.99760, -114.07617 50.9...",urban_high
2,landuse,retail,Dalhousie Station,"POLYGON ((-114.16421 51.10555, -114.16240 51.1...",urban_high
3,landuse,retail,Crowchild Square,"POLYGON ((-114.16627 51.10372, -114.16589 51.1...",urban_high
4,landuse,retail,None,"POLYGON ((-114.15018 51.10117, -114.14980 51.1...",urban_high


In [30]:
osm_lulc.to_file("../data/processed/osm-lulc.gpkg",
                 driver="GPKG",
                 engine="pyogrio")

In [32]:
osm_lulc.lulc.unique()

array(['urban_high', 'urban_open', 'forest', 'cropland', 'rock_soil',
       'urban_low', 'grassland', 'water', 'wetland', 'snow_ice'],
      dtype=object)

In [ ]:
geom = osm_lulc.loc[osm_lulc.lulc == "urban_low"]
geom = geom.to_json()
geo_j = folium.GeoJson(geom)
m = folium.Map(tiles="CartoDB Positron")
geo_j.add_to(m)
m

In [ ]:
bbox=(53.9, 53.3, -113.0, -113.9)
osm_dfs = list()

for key in schema.key.unique():
    values = schema.loc[schema.key == key, "value"].unique()
    tags = {"building": list(values)}
    df = ox.features_from_bbox(bbox=bbox, tags=tags)
    df.reset_index(inplace=True)
    df = osm.loc[df.element_type.isin(["relation", "way"]), :]
    osm_dfs.append(df)